In [1]:
import pandas as pd
import numpy as py

#Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

#Regression/Modelling Libraries
from scipy.stats import linregress
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

#Load Data
import sklearn.datasets as dta
import re

In [6]:
# constatnts
INPUT_CSV_PATH = "DOHMH_Dog_Bite_Data.csv"
DEFAULT_BREED = "unknown"

In [8]:
# Load the restaurant ethnicities into a DataFrame
dog_df = pd.read_csv(INPUT_CSV_PATH)

# Display sample data
dog_df.head()

,UniqueID,DateOfBite,Species,Breed,Age,Gender,SpayNeuter,Borough,ZipCode
0,1,January 01 2018,DOG,UNKNOWN,NaN,U,False,Brooklyn,11220
1,2,January 04 2018,DOG,UNKNOWN,NaN,U,False,Brooklyn,NaN
2,3,January 06 2018,DOG,Pit Bull,NaN,U,False,Brooklyn,11224
3,4,January 08 2018,DOG,Mixed/Other,4,M,False,Brooklyn,11231
4,5,January 09 2018,DOG,Pit Bull,NaN,U,False,Brooklyn,11224


# AGE

In [9]:
# age, clean and convert
# Used XPERT to get ideas on how to tackle age.  There were several types of data scrubbing needed for age with this dataset.
def convert_age_to_decimal(age_str):
    age_str = str(age_str)
    week_match = re.search(r'(\d+)\s*(WKS|W|WEEK|WEEKS)', age_str)
    month_match = re.search(r'(\d+)\s*(MTHS|M|MTH|MONTH)', age_str)
    year_match = re.search(r'(\d+)\s*(YRS|Y|YR|YEAR)', age_str)
    combined_match = re.search(r'(\d+)\s*(&)', age_str)    # Extract Combined Year and Month data when & sign is used.
    
    if week_match:
        weeks = int(week_match.group(1))
        weeks = weeks/52
        decimal_age = weeks    
    elif month_match:
        months = int(month_match.group(1))
        months = months/12
        decimal_age = months
    elif year_match:
        years = int(year_match.group(1))
        decimal_age = years
    elif combined_match:
        yearmonth = re.split(r'\s*&\s*', age_str)
        year = int(yearmonth[0])
        month = int(yearmonth[1])/12
        yearmonth = year + month
        decimal_age = yearmonth
    else:
        # raise ValueError(f'No match for {age_str}')
        try:
            decimal_age = float(age_str)
        except ValueError:
            decimal_age = None
    return decimal_age

In [10]:
print(dog_df.Age.isna().count())
print(dog_df.Age.isna().sum())
print(dog_df.Age.isna().count() - dog_df.Age.isna().sum())

22663
11221
11442


In [11]:
dog_df['age_decimal'] = dog_df['Age'].map(convert_age_to_decimal)

In [12]:
print(dog_df.age_decimal.isna().count())
print(dog_df.age_decimal.isna().sum())
print(dog_df.age_decimal.isna().count() - dog_df.age_decimal.isna().sum())

22663
11276
11387


In [13]:
dog_df.dropna(subset=['age_decimal'], inplace=True)

In [14]:
print(dog_df.age_decimal.isna().count())
print(dog_df.age_decimal.isna().sum())
print(dog_df.age_decimal.isna().count() - dog_df.age_decimal.isna().sum())

11387
0
11387


# Dates and Times

In [15]:
# found a '?' value in the ZipCode column, so eliminating that row from the data set.

dog_df = dog_df.loc[dog_df["ZipCode"] != '?']
dog_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11387 entries, 3 to 22660
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   UniqueID     11387 non-null  int64  
 1   DateOfBite   11387 non-null  object 
 2   Species      11387 non-null  object 
 3   Breed        11298 non-null  object 
 4   Age          11387 non-null  object 
 5   Gender       11387 non-null  object 
 6   SpayNeuter   11387 non-null  bool   
 7   Borough      11387 non-null  object 
 8   ZipCode      9917 non-null   object 
 9   age_decimal  11387 non-null  float64
dtypes: bool(1), float64(1), int64(1), object(7)
memory usage: 900.7+ KB


In [16]:
#Changing the datatype for the DateOfBite column from text to datetime.

dog_df['DateOfBite'] = pd.to_datetime(dog_df['DateOfBite'], errors='coerce')
dog_df['DateOfBite'][:5]

3    2018-01-08
5    2018-01-03
8    2018-01-04
9    2018-01-10
11   2018-01-07
Name: DateOfBite, dtype: datetime64[ns]

In [17]:
# split to year, month, day, dayofweek, weekname and droped the DateOFBite, Species column.

dog_df['Year'] = dog_df['DateOfBite'].dt.year
dog_df['Month'] = dog_df['DateOfBite'].dt.month
dog_df['Day'] = dog_df['DateOfBite'].dt.day 
dog_df['DayOfWeek'] = dog_df['DateOfBite'].dt.dayofweek # day of the week starts(Mon = 0)
dog_df= dog_df.drop(['Species'], axis=1)
dog_df.head()

,UniqueID,DateOfBite,Breed,Age,Gender,SpayNeuter,Borough,ZipCode,age_decimal,Year,Month,Day,DayOfWeek
3,4,2018-01-08,Mixed/Other,4,M,False,Brooklyn,11231,4.0,2018,1,8,0
5,6,2018-01-03,BASENJI,4Y,M,False,Brooklyn,11231,4.0,2018,1,3,2
8,9,2018-01-04,American Pit Bull Mix / Pit Bull Mix,5Y,M,False,Brooklyn,11235,5.0,2018,1,4,3
9,10,2018-01-10,MIXED,3Y,F,False,Brooklyn,11208,3.0,2018,1,10,2
11,12,2018-01-07,Yorkshire Terrier Crossbreed,7,M,True,Brooklyn,11208,7.0,2018,1,7,6


In [18]:
# Create a dictionary mapping months to seasons

season = {12: 'Winter', 1: 'Winter', 2: 'Winter', 3: 'Spring', 4: 'Spring', 5: 'Spring',
           6: 'Summer', 7: 'Summer', 8: 'Summer', 9: 'Fall', 10: 'Fall', 11: 'Fall'}

# Create a new column 'Seasons' based on the mapping

dog_df['Seasons'] = dog_df['Month'].map(season)

dog_df.head()

,UniqueID,DateOfBite,Breed,Age,Gender,SpayNeuter,Borough,ZipCode,age_decimal,Year,Month,Day,DayOfWeek,Seasons
3,4,2018-01-08,Mixed/Other,4,M,False,Brooklyn,11231,4.0,2018,1,8,0,Winter
5,6,2018-01-03,BASENJI,4Y,M,False,Brooklyn,11231,4.0,2018,1,3,2,Winter
8,9,2018-01-04,American Pit Bull Mix / Pit Bull Mix,5Y,M,False,Brooklyn,11235,5.0,2018,1,4,3,Winter
9,10,2018-01-10,MIXED,3Y,F,False,Brooklyn,11208,3.0,2018,1,10,2,Winter
11,12,2018-01-07,Yorkshire Terrier Crossbreed,7,M,True,Brooklyn,11208,7.0,2018,1,7,6,Winter


In [19]:
# Create a dictionary mapping dayofweek to seasons
weekname = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday',
           6: 'Sunday'}

# Create a new column 'season' based on the mapping
dog_df['DayOFWeek'] = dog_df['DayOfWeek'].map(weekname)

dog_df.head()

,UniqueID,DateOfBite,Breed,Age,Gender,SpayNeuter,Borough,ZipCode,age_decimal,Year,Month,Day,DayOfWeek,Seasons,DayOFWeek
3,4,2018-01-08,Mixed/Other,4,M,False,Brooklyn,11231,4.0,2018,1,8,0,Winter,Monday
5,6,2018-01-03,BASENJI,4Y,M,False,Brooklyn,11231,4.0,2018,1,3,2,Winter,Wednesday
8,9,2018-01-04,American Pit Bull Mix / Pit Bull Mix,5Y,M,False,Brooklyn,11235,5.0,2018,1,4,3,Winter,Thursday
9,10,2018-01-10,MIXED,3Y,F,False,Brooklyn,11208,3.0,2018,1,10,2,Winter,Wednesday
11,12,2018-01-07,Yorkshire Terrier Crossbreed,7,M,True,Brooklyn,11208,7.0,2018,1,7,6,Winter,Sunday


## BREED

In [20]:
def column_info(df, col):
    print(f'nunique: {df[col].nunique()}')
    print(f'count: {df[col].count()}')
    print(df[col].value_counts().head(22))

In [21]:
# mixes with consildated breeds
def consolidated_breed_mix_removal(execute=True):
    for target in breed_consolidation:
        contains_target_mask = dog_df.breed_norm.str.contains(target)
        not_target_mask = dog_df.breed_norm != target
        to_replace_mask = contains_target_mask & not_target_mask
        if execute:
            dog_df.loc[to_replace_mask, 'breed_norm'] = DEFAULT_BREED
        else:
            print(dog_df.loc[to_replace_mask, 'breed_norm'].values)

In [22]:
dog_df.Breed = dog_df.Breed.fillna(DEFAULT_BREED)

In [23]:
dog_df['breed_norm'] = dog_df['Breed'].str.lower()
dog_df.breed_norm.replace
column_info(dog_df, 'breed_norm')

nunique: 1434
count: 11387
breed_norm
pit bull                                1410
shih tzu                                 473
american pit bull mix / pit bull mix     453
unknown                                  392
german shepherd                          362
mixed/other                              338
chihuahua                                323
mixed breed                              314
american pit bull terrier/pit bull       299
yorkshire terrier                        286
mixed                                    247
maltese                                  227
siberian husky                           174
labrador retriever                       158
rottweiler                               141
jack russ                                135
labrador retriever crossbreed            135
poodle, standard                         113
cocker spaniel                           112
shiba inu                                110
american staffordshire terrier           109
bull dog, english

In [24]:
breed_res = {
    re.compile(r".*pit.?bull.*", re.IGNORECASE): 'pit bull',
    re.compile(r".*mix.*", re.IGNORECASE): DEFAULT_BREED,
    re.compile(r".*/.*", re.IGNORECASE): DEFAULT_BREED,
    re.compile(r".*\s+x.*", re.IGNORECASE): DEFAULT_BREED,
    re.compile(r".*-x.*", re.IGNORECASE): DEFAULT_BREED,
    re.compile(r".*mxied.*", re.IGNORECASE): DEFAULT_BREED,
    re.compile(r".*boxer beagle.*", re.IGNORECASE): DEFAULT_BREED,
    re.compile(r".*border collile, rotweiller, lab, boxer.*", re.IGNORECASE): DEFAULT_BREED,
    re.compile(r".*exotic american bully.*", re.IGNORECASE): 'pit bull',
    re.compile(r".*bord.*x.*", re.IGNORECASE): 'french mastiff',
    re.compile(r".*&.*", re.IGNORECASE): DEFAULT_BREED,
    re.compile(r".*cross.*", re.IGNORECASE): DEFAULT_BREED,
    # order may be important after this point
    re.compile(r".*collie.*", re.IGNORECASE): 'collie',
     # these patterns were developed after rare breeds removal
}
for breed_re in breed_res:
    dog_df.breed_norm = dog_df['breed_norm'].str.replace(breed_re, breed_res[breed_re], regex=True)
column_info(dog_df, 'breed_norm')

nunique: 388
count: 11387
breed_norm
unknown                           3460
pit bull                          2500
shih tzu                           473
german shepherd                    362
chihuahua                          323
yorkshire terrier                  286
maltese                            227
siberian husky                     174
labrador retriever                 158
rottweiler                         141
jack russ                          135
poodle, standard                   113
cocker spaniel                     112
shiba inu                          110
american staffordshire terrier     109
bull dog, english                   92
beagle                              86
golden retriever                    84
bull dog                            80
bull dog, french                    79
pomeranian                          75
boxer                               57
Name: count, dtype: int64


In [25]:
breed_consolidation = {
    'jack russ': ['jack russ', 'jack russell terrier', 'terrier, jack russ'],
    DEFAULT_BREED: ['labrador boxer'],
    'poodle': ['poodle, miniature', 'poodle, standard', 'poodle, toy', 'poodle', 'large poodle', 
               'french poodle'],
    'poo': [],
    'doodle': [],
    'pit bull': ['american staffordshire terrier'],
    'retriever': ['labrador retriever', 'golden retriever', 'nova scotia duck tolling retriever', 'labarador retriever', 'chesapeake bay retriever'],
    'dachshund': ['doxxin','dachshund smooth coat', 'dachshund, wirehaired, miniature', 
                  'dachshund, wirehaired', 'dachshund smooth coat miniature','dachshund, long haired', 
                  'dachshund','dachshund, long haired miniature', 'dachshund, miniature', 
                  'dachshund miniature', 'daschound', 'daschund'],
     # these patterns were developed after rare breeds removal
    'bull dog': ['bull dog, english', 'bull dog', 'bull dog, french'],
}
for target, patterns in breed_consolidation.items():
    for pat in patterns:
        # print(f'replacing {pat} with {target}')
        dog_df.breed_norm = dog_df.breed_norm.str.replace(pat, target)
consolidated_breed_mix_removal(execute=True)
print(dog_df['breed_norm'].count())
column_info(dog_df, 'breed_norm')

11387
nunique: 320
count: 11387
breed_norm
unknown              3819
pit bull             2609
shih tzu              473
german shepherd       362
chihuahua             323
yorkshire terrier     286
bull dog              251
retriever             249
maltese               227
siberian husky        174
rottweiler            141
jack russ             138
cocker spaniel        112
shiba inu             110
dachshund              95
beagle                 86
pomeranian             75
bichon frise           57
boxer                  57
boston terrier         56
havanese               54
cane corso             54
Name: count, dtype: int64


### due to project time constraints we had to do this rather than continue to add more patterns

In [26]:
rare_breeds = ['boxer', 'bichon frise', 'boston terrier', 'havanese', 'chow chow',
       'cane corso', 'australian shepherd', 'doberman pinscher',
       'lhasa apso', 'american bully', 'miniature pinscher', 'pekingese',
       'akita', 'staffordshire bull terrier', 'west high white terrier',
       'collie', 'terrier', 'bull terrier', 'wheaton terrier',
       'schnauzer, miniature', 'puggle', 'belgian malinois', 'great dane',
       'rhodesian ridgeback', 'mastiff, bull', 'pug', 'corgi',
       'american eskimo', 'shar-pei, chinese', 'mastiff', 'mutt',
       'schnauzer, standard', 'rat terrier', 'cairn terrier',
       'jindo dog,', 'weimaraner', 'labrador', 'husky', 'morkie',
       'basenji', 'australian cattle dog', 'hound', 'cotton de tulear',
       'scottish terrier', 'bernese mountain', 'dalmatian', 'vizsla',
       'portuguese water dog', 'silky terrier', 'saint bernard',
       'border terrier', 'greyhound', 'tibetan terrier', 'newfoundland',
       'brussels griffon', 'alaskan klee kai', 'basset hound',
       'welsh corgi, pembroke', 'american bull dog', 'american bulldog',
       'bulldog', 'cavalier king charles spaniel', 'samoyed',
       'brittany spaniel', 'mastiff, tibetan', 'norfolk terrier', 'bully',
       'pointer, german shorthaired', 'mini australian shepherd',
       'french mastiff', 'pointer', 'nova scotia duck tolling retriever',
       'german pinscher', 'papillon', 'mastiff, old english',
       'toy fox terrier', 'english springer spaniel',
       'english cocker spaniel', 'unsure', 'welsh corgi, cardigan',
       'airedale terrier', 'springer spaniel', 'cocker spaniel, american',
       'black mouth cur', 'australian cattle', 'great pyrenees',
       'wire fox terrier', 'pomski', 'kerry blue terrier', 'schnoodle',
       'bloodhound', 'sheperd', 'belgian shepherd',
       'coonhound, black and tan', 'australian cattledog', 'plott hound',
       'staffordshire', 'schnauzer, giant', 'bouvier des flandres',
       'pomsky', 'chinese crested', 'catahoula leopard', 'schnauzer',
       'clumber spaniel', 'coonhound, treeing walker', 'yorkie',
       'shepherd', 'blue tick hound', 'mastiff, neapolitan',
       'australian silky terrier', 'pharoh hound', 'english bull terrier',
       'belgian sheepdog', 'potcake', 'carolina dog',
       'black russian terrier', 'patterdale terrier',
       'glen of imaal terrier', 'catahoula leopard dog',
       'alaskan malamute', 'sheepdog', 'lowchen', 'shetland sheepdog',
       'victorian bulldog', 'alaskan husky', 'golden doddle',
       'english bull dog', 'mini bull terrier', 'cattle dog',
       'medium size dog', 'zuchon', 'norwich terrier', 'french bull dog',
       'unkown', 'dandie dinmont terrier', 'stafford', 'pocket bully',
       'english bulldog terrier', 'hatchie', 'russian terrier',
       'large dog', 'wirehaired pointing griffon', 'borzoi',
       'bulldog, american', 'husky labrador',
       'portuguese podengo pequeno', 'mini pincher',
       'nova scotia duck tollier', 'chinese crested hairless',
       'felky terrier', 'welsh springer spaniel', 'west highland terr',
       'potcake dog', 'shichon', 'puli', 'australian sheep dog',
       'boston border terrier', 'alaskan malmute', 'catahoula cur',
       'teacup maltese', 'shichi', 'queensland blue healer',
       'belgium shepard', 'sharpei', 'australian shpherd',
       'pocket beagle', 'not sure, rescue', 'bolognese bichon',
       'english bulldog', 'red nose pit', 'staffordshire terrier',
       'pomeranian english spaniel', 'parson terr', 'irish terrier',
       'schipperkee', 'irish wolfhound', 'feist', 'miki',
       'parson russell terrieer', 'cavashu', 'chinese tien yun chu',
       'marquis', 'maltese terrier', 'central asian shepherd',
       'malti tzu', 'dogo', 'yorkie chon', 'english sheepdog',
       'toy yorkie', 'american bull', 'russell terr', 'boerboel',
       'malshi', 'beagle hound', 'russell terrier', 'italian masiff',
       'italian masif', 'laborador mastiff', 'markey',
       'king charles cavlier', 'malamute', 'parson russell terrier',
       'bully breed', 'amer bull terrier', 'white point griffon',
       'mini golden doddle', 'dogo argentino', 'briard', 'englih bulldog',
       'african boerboel', 'sheep dog', 'tibetan mastiff',
       'multi collar terrier', 'english mastiff', 'italian king corso',
       'coton de tulear', 'westies', 'belgian turvuren', 'westy',
       'alapaha bulldog', 'persa canario', 'english setter',
       'miniature schnauzer', 'aust cattle dog', 'harrier hound',
       'labarador retriever', 'australian terrier', 'english toy spaniel',
       'mini golden retrever', 'catahoula', 'unnkown', 'big size dog',
       'german wirehaired pointer', 'austrailian shepard', 'hot dog',
       'blue healer', 'king korso', 'corgie', 'small dog', 'lagotte',
       'king charles', 'japanese shiba', 'lapso',
       'miniature australian sheperd', 'petit basset griffon vendeen',
       '2 bull terrier dogs', 'peagle', 'shiloh shepherd',
       'miniature american shepard', 'japanese spitz', 'small',
       'lakeland terrier', 'asian sheperd', 'sapsal', 'italian greyhound',
       'teacup chi hua hua', 'russian wolfhound',
       'english staffordshire bull terrier', 'uncertain',
       'terrier, kerry blue', 'shepard', 'english staffordshire terrier',
       'shi-po', 'medium breed', 'shih tzu pug', 'boykin spaniel',
       'english pointer', 'mexican hairless', 'pitbll terrier',
       'vizsla breed', 'arubian cunucu', 'chinese crested powder puff',
       'wire hair terrier', 'mini australian shpherd', 'english bullbog',
       'british bulldog', 'walkerhound', 'german shepard',
       'spinone italiano', 'large', 'fox hound', 'italian mastiff',
       'parson russellterr', 'staffordshire terr', 'shorky',
       'sherlyn shepherd', 'eng setter', 'korean sapsali',
       'doberman and labrador', 'english brindle', 'cesky terrier',
       'dandie dinmont', 'australian blue heeler', 'llewelin setter',
       'soft coated wheaten terrier', 'chesapeake bay retriever',
       'westie terrier', 'beauceron', 'american hairless terrier',
       'american bull terrier', 'spanish water dog',
       'mini american husky', 'saluki', 'american coonhound',
       'tibetan spaniel', 'blue lacy', 'whippet', 'soft wheaton terrier',
       'micro bully', 'dunker']
rare_breed_mask = dog_df.breed_norm.isin(rare_breeds)
dog_df.loc[rare_breed_mask, 'breed_norm'] = 'tbd'
column_info(dog_df, 'breed_norm')

nunique: 18
count: 11387
breed_norm
unknown              3819
pit bull             2609
tbd                  1857
shih tzu              473
german shepherd       362
chihuahua             323
yorkshire terrier     286
bull dog              251
retriever             249
maltese               227
siberian husky        174
rottweiler            141
jack russ             138
cocker spaniel        112
shiba inu             110
dachshund              95
beagle                 86
pomeranian             75
Name: count, dtype: int64


In [27]:
dog_df.head(20)

,UniqueID,DateOfBite,Breed,Age,Gender,SpayNeuter,Borough,ZipCode,age_decimal,Year,Month,Day,DayOfWeek,Seasons,DayOFWeek,breed_norm
3,4,2018-01-08,Mixed/Other,4,M,False,Brooklyn,11231,4.0,2018,1,8,0,Winter,Monday,unknown
5,6,2018-01-03,BASENJI,4Y,M,False,Brooklyn,11231,4.0,2018,1,3,2,Winter,Wednesday,tbd
8,9,2018-01-04,American Pit Bull Mix / Pit Bull Mix,5Y,M,False,Brooklyn,11235,5.0,2018,1,4,3,Winter,Thursday,pit bull
9,10,2018-01-10,MIXED,3Y,F,False,Brooklyn,11208,3.0,2018,1,10,2,Winter,Wednesday,unknown
11,12,2018-01-07,Yorkshire Terrier Crossbreed,7,M,True,Brooklyn,11208,7.0,2018,1,7,6,Winter,Sunday,unknown
12,13,2018-01-06,Great Dane,6,F,True,Brooklyn,NaN,6.0,2018,1,6,5,Winter,Saturday,tbd
14,15,2018-01-09,Labradoodle,5,F,True,Brooklyn,11238,5.0,2018,1,9,1,Winter,Tuesday,unknown
18,19,2018-01-13,MIXED,7,M,False,Brooklyn,11237,7.0,2018,1,13,5,Winter,Saturday,unknown
20,21,2018-01-15,"Poodle, Miniature",8,M,True,Brooklyn,11217,8.0,2018,1,15,0,Winter,Monday,unknown
21,22,2018-01-13,American Pit Bull Terrier/Pit Bull,11,F,True,Brooklyn,11236,11.0,2018,1,13,5,Winter,Saturday,pit bull


## Future breed 

In [31]:
# rare_mask = dog_df.breed_norm.value_counts() <= 75
# print(dog_df.breed_norm.value_counts()[rare_mask].sum())
# rare_breeds = dog_df.breed_norm.value_counts()[rare_mask].index
# print(len(rare_breeds))
# column_info(dog_df, 'breed_norm')

In [32]:
# dog_df.loc[dog_df.breed_norm != DEFAULT_BREED, 'Age'].count()

In [33]:
#dog_df.loc[~rare_mask]

In [34]:
# target = 'poodle'
# contains_target_mask = dog_df.breed_norm.str.contains(target)
# not_target_mask = dog_df.breed_norm != target
# to_replace_mask = contains_target_mask & not_target_mask
# dog_df.loc[to_replace_mask, 'breed_norm'] = DEFAULT_BREED

In [35]:
# #Pattern investigation cell
# breed_mask = dog_df.breed_norm.str.contains('bull')
# print(breed_mask.sum())
# dog_df.loc[breed_mask, 'breed_norm'].unique()

In [36]:
# #regex test cell
# test_re = re.compile(r".*retriever.*", re.IGNORECASE)
# test_mask = dog_df['breed_norm'].str.contains(test_re, regex=True)
# column_info(dog_df[test_mask], 'breed_norm')

In [ ]:
# dog_df.loc[test_mask, 'breed_norm'].unique()

In [ ]:
# dog_df.breed_norm = dog_df['breed_norm'].str.replace(test_re, DEFAULT_BREED, regex=True)
# column_info(dog_df, 'breed_norm')

## GENDER

In [42]:
Borough_dog=clean_df.groupby(["Borough"])["zipcode"].count()
Age_dog=clean_df.groupby(["Age"])["zipcode"].count()



data={
    "borough_dog": mean_tumor,
    "Median Tumor Volume" : median_tumor,
    "Tumor Volume Variance": var_tumor,
    "Tumor Volume Std.Dev.": std_dev_tumor,
    "Tumot Volume Std.Err.": std_err_tumor,  
     }
summary= pd.DataFrame(data)
summary


TypeError: unhashable type: 'Series'

In [ ]:
dog_df